In [1]:
import pandas as pd
import json
from matplotlib import pyplot as plt

In [2]:

df_transactions = pd.read_csv('data/transactions_data.csv')


In [3]:
mcc_codes = json.load(open('data/raw/mcc_codes.json'))

In [105]:

def cash_flow_summary(
    df: pd.DataFrame, client_id: int, start_date: str, end_date: str
) -> pd.DataFrame:
    """
    For the period defined by start_date and end_date (both inclusive), retrieve the available client data and return a Pandas DataFrame containing cash flow information.

    If the period exceeds 60 days, group the data by month, using the end of each month for the date. If the period is 60 days or shorter, group the data by week.

        The expected columns are:
            - Date --> the date for the period. YYYY-MM if period larger than 60 days, YYYY-MM-DD otherwise.
            - Inflows --> the sum of the earnings (positive amounts)
            - Outflows --> the sum of the expenses (absolute values of the negative amounts)
            - Net Cash Flow --> Inflows - Outflows
            - % Savings --> Percentage of Net Cash Flow / Inflows

        The DataFrame should be sorted by ascending date and values rounded to 2 decimals. The columns should be in the given order.

        Parameters
        ----------
        df : pandas DataFrame
           DataFrame  of the data to be used for the agent.
        client_id : int
            Id of the client.
        start_date : str
            Start date for the date period. In the format "YYYY-MM-DD".
        end_date : str
            End date for the date period. In the format "YYYY-MM-DD".


        Returns
        -------
        Pandas Dataframe with the cash flow summary.
    """
    if  df["date"].dtype != "datetime64[ns]":
        df["date"] = pd.to_datetime(df["date"])
    if df["amount"].dtype != "float64":
        df["amount"] = df["amount"].apply(lambda x: x.replace("$", "").replace(",", "")).astype(float)
        
    # Convert start and end dates to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    delta_days = (end_date - start_date).days

    # Set the period format based on the number of days
    order_by = "ME" if delta_days > 60 else "W"
    # Convert 'date' column to datetime if not already

    # Filter for the client ID and date range
    df = df[(df["client_id"] == client_id) & (df["date"] >= start_date) & (df["date"] <= end_date)]
    
    # group by the month 
    df = df.groupby(pd.Grouper(key="date", freq=order_by)).agg(
        Inflows=("amount", lambda x: x[x > 0].sum()),
        Outflows=("amount", lambda x: -x[x < 0].sum()),
    ).reset_index()

    # Calculate the Net Cash Flow
    df["Net Cash Flow"] = df["Inflows"] - df["Outflows"]
    df["% Savings"] = df["Net Cash Flow"] / df["Inflows"] *100

    # name
    df["Date"] = df["date"]
    # Drop the date column
    df = df.drop(columns="date")
    # order the columns
    df = df[["Date", "Inflows", "Outflows", "Net Cash Flow", "% Savings"]]
    # order the rows
    df = df.sort_values(by="Date", ascending=True)

    # 2 decimal places
    df = df.round(2)

    # date format string "YYYY-MM-DD"
    df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")  if order_by == "W" else df["Date"].dt.strftime("%Y-%m")
    return df

In [106]:
end_date = "2020-01-01"
start_date = "1800-01-31"
client_id = 1556
df_selected = cash_flow_summary(df_transactions, client_id, start_date, end_date)



In [107]:
expected_answer = pd.DataFrame(
            {
                "Date": [
                    "2013-01-06",
                    "2013-01-13",
                    "2013-01-20",
                    "2013-01-27",
                    "2013-02-03",
                    "2013-02-10",
                    "2013-02-17",
                    "2013-02-24",
                    "2013-03-03",
                ],
                "Inflows": [
                    432.74,
                    518.67,
                    297.63,
                    450.91,
                    863.16,
                    288.75,
                    466.82,
                    1019.02,
                    138.35,
                ],
                "Outflows": [129.0, 0.0, 66.0, 0.0, 156.0, 0.0, 0.0, 71.0, 0.0],
                "Net Cash Flow": [
                    303.74,
                    518.67,
                    231.63,
                    450.91,
                    707.16,
                    288.75,
                    466.82,
                    948.02,
                    138.35,
                ],
                "% Savings": [
                    70.19,
                    100.0,
                    77.82,
                    100.0,
                    81.93,
                    100.0,
                    100.0,
                    93.03,
                    100.0,
                ],
            }
        )


In [95]:
# from data.data_functions import cash_flow_summary

In [96]:
answer = cash_flow_summary(df_transactions, 126, "2013-01-01", "2013-02-28")

In [97]:
answer.Date.values

array(['2013-01-06', '2013-01-13', '2013-01-20', '2013-01-27',
       '2013-02-03', '2013-02-10', '2013-02-17', '2013-02-24',
       '2013-03-03'], dtype=object)

In [98]:
answer

,Date,Inflows,Outflows,Net Cash Flow,% Savings
0,2013-01-06,432.74,129.0,303.74,70.19
1,2013-01-13,518.67,-0.0,518.67,100.00
2,2013-01-20,297.63,66.0,231.63,77.82
3,2013-01-27,450.91,-0.0,450.91,100.00
4,2013-02-03,863.16,156.0,707.16,81.93
5,2013-02-10,288.75,-0.0,288.75,100.00
6,2013-02-17,466.82,-0.0,466.82,100.00
7,2013-02-24,1019.02,71.0,948.02,93.03
8,2013-03-03,138.35,-0.0,138.35,100.00


In [99]:
expected_answer

,Date,Inflows,Outflows,Net Cash Flow,% Savings
0,2013-01-06,432.74,129.0,303.74,70.19
1,2013-01-13,518.67,0.0,518.67,100.00
2,2013-01-20,297.63,66.0,231.63,77.82
3,2013-01-27,450.91,0.0,450.91,100.00
4,2013-02-03,863.16,156.0,707.16,81.93
5,2013-02-10,288.75,0.0,288.75,100.00
6,2013-02-17,466.82,0.0,466.82,100.00
7,2013-02-24,1019.02,71.0,948.02,93.03
8,2013-03-03,138.35,0.0,138.35,100.00


In [100]:
pd.testing.assert_frame_equal(answer, expected_answer)


In [101]:
answer = cash_flow_summary(df_transactions, 50, "2011-01-01", "2011-04-30")
expected_answer = pd.DataFrame(
    {
        "Date": ["2011-01", "2011-02", "2011-03", "2011-04"],
        "Inflows": [2488.64, 2154.61, 2721.85, 2725.5],
        "Outflows": [183.0, 578.0, 628.0, 483.0],
        "Net Cash Flow": [2305.64, 1576.61, 2093.85, 2242.5],
        "% Savings": [92.65, 73.17, 76.93, 82.28],
    }
)

/tmp/ipykernel_11593/1226288081.py:52: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.groupby(pd.Grouper(key="date", freq=order_by)).agg(


In [102]:
answer

,Date,Inflows,Outflows,Net Cash Flow,% Savings
0,2011-01,2488.64,183.0,2305.64,92.65
1,2011-02,2154.61,578.0,1576.61,73.17
2,2011-03,2721.85,628.0,2093.85,76.93
3,2011-04,2725.50,483.0,2242.50,82.28


In [103]:
expected_answer

,Date,Inflows,Outflows,Net Cash Flow,% Savings
0,2011-01,2488.64,183.0,2305.64,92.65
1,2011-02,2154.61,578.0,1576.61,73.17
2,2011-03,2721.85,628.0,2093.85,76.93
3,2011-04,2725.50,483.0,2242.50,82.28


In [104]:
pd.testing.assert_frame_equal(answer, expected_answer)
